# Effect of AI on Customer Churn
## Predicting Customer Churn

### Import Libraries

In [21]:
# import libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline 

### Import Dataset

In [7]:
# import train and test files
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [8]:
train.head()

,ID,Age,AI_Interaction_Level,Satisfaction_with_AI_Services,AI_Personalization_Effectiveness,AI_Response_Time,Overall_Usage_Frequency,Customer_Service_Interactions,Change_in_Usage_Patterns,Customer_Churn
0,1,59,2,7,7,2,18,5,2,1
1,2,49,2,6,5,1,3,2,0,0
2,3,35,2,3,3,3,8,2,1,1
3,4,63,2,3,3,2,6,6,2,1
4,5,28,4,6,6,1,11,3,-1,0


In [9]:
test.head()

,ID,Age,AI_Interaction_Level,Satisfaction_with_AI_Services,AI_Personalization_Effectiveness,AI_Response_Time,Overall_Usage_Frequency,Customer_Service_Interactions,Change_in_Usage_Patterns,Customer_Churn
0,1,59,1,2,3,1,11,5,-1,1
1,2,49,3,4,3,2,2,0,2,1
2,3,35,3,7,6,2,22,4,0,1
3,4,63,2,8,8,1,25,4,-1,1
4,5,28,4,5,5,2,14,1,1,1


### Data Exploration

#### Data dimensions, feature names and feature types

In [11]:
train.shape

(5000, 10)

In [12]:
train.columns

Index(['ID', 'Age', 'AI_Interaction_Level', 'Satisfaction_with_AI_Services',
       'AI_Personalization_Effectiveness', 'AI_Response_Time',
       'Overall_Usage_Frequency', 'Customer_Service_Interactions',
       'Change_in_Usage_Patterns', 'Customer_Churn'],
      dtype='object')

In [13]:
train.dtypes

ID                                  int64
Age                                 int64
AI_Interaction_Level                int64
Satisfaction_with_AI_Services       int64
AI_Personalization_Effectiveness    int64
AI_Response_Time                    int64
Overall_Usage_Frequency             int64
Customer_Service_Interactions       int64
Change_in_Usage_Patterns            int64
Customer_Churn                      int64
dtype: object

In [18]:
train['Customer_Churn'].value_counts(normalize=True)

Customer_Churn
0    0.5818
1    0.4182
Name: proportion, dtype: float64

In [20]:
train.describe(include='all')

,ID,Age,AI_Interaction_Level,Satisfaction_with_AI_Services,AI_Personalization_Effectiveness,AI_Response_Time,Overall_Usage_Frequency,Customer_Service_Interactions,Change_in_Usage_Patterns,Customer_Churn
count,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,2500.500000,42.575000,2.468800,5.51060,5.525800,1.981600,15.515000,5.066200,-0.005000,0.418200
std,1443.520003,12.554597,0.940853,2.86446,2.884129,0.815227,8.703018,3.178402,1.421049,0.493313
min,1.000000,21.000000,1.000000,1.00000,1.000000,1.000000,1.000000,0.000000,-2.000000,0.000000
25%,1250.750000,32.000000,2.000000,3.00000,3.000000,1.000000,8.000000,2.000000,-1.000000,0.000000
50%,2500.500000,43.000000,2.000000,5.00000,5.000000,2.000000,16.000000,5.000000,0.000000,0.000000
75%,3750.250000,53.000000,3.000000,8.00000,8.000000,3.000000,23.000000,8.000000,1.000000,1.000000
max,5000.000000,64.000000,4.000000,10.00000,10.000000,3.000000,30.000000,10.000000,2.000000,1.000000


### Data Wrangling and Visualization